In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from IPython.core.pylabtools import figsize

In [2]:
rawtext = open("/media/joe/HD/data/gutenberg/alice_in_wonderland.txt", "r").read()[710:]
len(rawtext)

172656

In [3]:
split = int(0.9*len(rawtext))
trainraw = rawtext.lower()[:split]
testraw = rawtext.lower()[split:]

In [4]:
chars = sorted(list(set(rawtext.lower())))
charmap = {chars[i]:i for i in range(len(chars))}
len(chars)

60

In [5]:
seq_len = 25

def train_inpt_fn():
    def _gen():
        while True:
            seq_start = np.random.randint(0, len(trainraw)-seq_len-1)
            sequence = trainraw[seq_start:seq_start+seq_len]
            yield {"x":np.array([charmap[c] for c in sequence], dtype=np.int32)}, \
            charmap[trainraw[seq_start+seq_len]]
            
    ds = tf.data.Dataset.from_generator(_gen, ({"x":tf.int32}, tf.int32), 
                                        ({"x":[seq_len,]}, []))
    ds = ds.batch(1024)
    ds = ds.prefetch(1)
    return ds.make_one_shot_iterator().get_next()

def test_inpt_fn():
    def _gen():
        for i in range(1000):
            seq_start = i*seq_len
            sequence = testraw[seq_start:seq_start+seq_len]
            yield {"x":np.array([charmap[c] for c in sequence], dtype=np.int32)}, \
                  charmap[testraw[seq_start+seq_len]]
            
    ds = tf.data.Dataset.from_generator(_gen, ({"x":tf.int32}, tf.int32), 
                                        ({"x":[seq_len,]}, []))
    ds = ds.batch(512)
    ds = ds.prefetch(1)
    return ds.make_one_shot_iterator().get_next()

In [6]:
train_inpt_fn()

({'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 25) dtype=int32>},
 <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>)

In [7]:
def model_fn(features, labels, mode, params):
    
    x_oh = tf.one_hot(features["x"], params["num_chars"])
    if labels is not None:
        y_oh = tf.one_hot(labels, params["num_chars"])

    if params["lstm"]:
        cell = tf.nn.rnn_cell.LSTMCell(params["hidden"])
    else:
        cell = tf.nn.rnn_cell.BasicRNNCell(params["hidden"])
    outputs, state = tf.nn.dynamic_rnn(cell, x_oh, dtype=tf.float32)
    net = outputs[:, -1, :]
    
    is_training = mode == tf.estimator.ModeKeys.TRAIN

    net = tf.layers.dense(net, params["hidden"], activation=tf.nn.relu)
    net = tf.layers.dropout(net, 0.5, training=is_training)

    logits = tf.layers.dense(net, params["num_chars"])
    probs = tf.nn.softmax(logits)
    predictions = tf.argmax(probs, -1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, 
                            predictions={"probs":probs},
                            export_outputs={"probs":tf.estimator.export.PredictOutput(probs)})
    
    loss = tf.losses.softmax_cross_entropy(y_oh, logits)
    accuracy = tf.metrics.accuracy(labels, predictions)
    metrics = {"accuracy":accuracy}
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode,
                            loss=loss, eval_metric_ops=metrics)
    
    gs = tf.train.get_or_create_global_step()
    #optimizer = tf.train.MomentumOptimizer(1e-3, 0.9)
    optimizer = tf.train.AdamOptimizer(1e-3)
    
    train_op = optimizer.minimize(loss, global_step=gs)
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, 
                        train_op=train_op, eval_metric_ops=metrics)

In [8]:
params={"num_chars":len(chars), "lstm":False, "hidden":256}
mod = "/media/joe/HD/text_pred_models/alice_in_wonderland_rnn_256_adam/"
model = tf.estimator.Estimator(model_fn, mod, 
                               params=params)

model.train(lambda: train_inpt_fn(), steps=25000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/media/joe/HD/text_pred_models/alice_in_wonderland_rnn_256_adam/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4fb65b1d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints 

INFO:tensorflow:global_step/sec: 4.58942
INFO:tensorflow:loss = 1.33031, step = 7100 (21.790 sec)
INFO:tensorflow:global_step/sec: 4.40715
INFO:tensorflow:loss = 1.34944, step = 7200 (22.693 sec)
INFO:tensorflow:global_step/sec: 4.12097
INFO:tensorflow:loss = 1.31265, step = 7300 (24.264 sec)
INFO:tensorflow:global_step/sec: 5.0673
INFO:tensorflow:loss = 1.27115, step = 7400 (19.735 sec)
INFO:tensorflow:global_step/sec: 5.29725
INFO:tensorflow:loss = 1.27938, step = 7500 (18.877 sec)
INFO:tensorflow:global_step/sec: 5.27889
INFO:tensorflow:loss = 1.2569, step = 7600 (18.944 sec)
INFO:tensorflow:global_step/sec: 4.81923
INFO:tensorflow:loss = 1.25773, step = 7700 (20.750 sec)
INFO:tensorflow:global_step/sec: 4.52329
INFO:tensorflow:loss = 1.24701, step = 7800 (22.113 sec)
INFO:tensorflow:global_step/sec: 4.51095
INFO:tensorflow:loss = 1.25344, step = 7900 (22.168 sec)
INFO:tensorflow:global_step/sec: 4.67317
INFO:tensorflow:loss = 1.21831, step = 8000 (21.396 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 4.79313
INFO:tensorflow:loss = 1.13431, step = 15100 (20.864 sec)
INFO:tensorflow:global_step/sec: 4.96483
INFO:tensorflow:loss = 1.12346, step = 15200 (20.142 sec)
INFO:tensorflow:global_step/sec: 4.98724
INFO:tensorflow:loss = 1.12556, step = 15300 (20.051 sec)
INFO:tensorflow:global_step/sec: 5.25434
INFO:tensorflow:loss = 1.1177, step = 15400 (19.032 sec)
INFO:tensorflow:global_step/sec: 5.20876
INFO:tensorflow:loss = 1.09952, step = 15500 (19.198 sec)
INFO:tensorflow:global_step/sec: 5.24096
INFO:tensorflow:loss = 1.10251, step = 15600 (19.079 sec)
INFO:tensorflow:global_step/sec: 5.25484
INFO:tensorflow:loss = 1.16618, step = 15700 (19.030 sec)
INFO:tensorflow:global_step/sec: 5.56013
INFO:tensorflow:loss = 1.08977, step = 15800 (17.985 sec)
INFO:tensorflow:global_step/sec: 5.70059
INFO:tensorflow:loss = 1.15122, step = 15900 (17.543 sec)
INFO:tensorflow:global_step/sec: 5.37088
INFO:tensorflow:loss = 1.13986, step = 16000 (18.619 sec)
INFO:tensor

INFO:tensorflow:loss = 1.08828, step = 23100 (21.397 sec)
INFO:tensorflow:global_step/sec: 4.76506
INFO:tensorflow:loss = 1.07566, step = 23200 (20.988 sec)
INFO:tensorflow:global_step/sec: 4.81662
INFO:tensorflow:loss = 1.00767, step = 23300 (20.762 sec)
INFO:tensorflow:global_step/sec: 4.94146
INFO:tensorflow:loss = 1.07963, step = 23400 (20.236 sec)
INFO:tensorflow:global_step/sec: 4.86381
INFO:tensorflow:loss = 1.02445, step = 23500 (20.559 sec)
INFO:tensorflow:global_step/sec: 4.69582
INFO:tensorflow:loss = 1.08697, step = 23600 (21.299 sec)
INFO:tensorflow:Saving checkpoints for 23609 into /media/joe/HD/text_pred_models/alice_in_wonderland_rnn_256_adam/model.ckpt.
INFO:tensorflow:global_step/sec: 4.30397
INFO:tensorflow:loss = 1.0182, step = 23700 (23.232 sec)
INFO:tensorflow:global_step/sec: 4.58509
INFO:tensorflow:loss = 1.06251, step = 23800 (21.812 sec)
INFO:tensorflow:global_step/sec: 4.21648
INFO:tensorflow:loss = 1.07444, step = 23900 (23.717 sec)
INFO:tensorflow:global_st

In [9]:
def serving_input_fn():
    features = {"x":tf.placeholder(tf.int32, [1, seq_len])}
    return tf.estimator.export.build_raw_serving_input_receiver_fn(features, None)

In [10]:
model.export_savedmodel("/media/joe/HD/text_pred_models/alice_rnn_256_adam_export/", serving_input_fn())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['probs', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /media/joe/HD/text_pred_models/alice_in_wonderland_rnn_256_adam/model.ckpt-25000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /media/joe/HD/text_pred_models/alice_rnn_256_adam_export/temp-b'1536113383'/saved_model.pb


b'/media/joe/HD/text_pred_models/alice_rnn_256_adam_export/1536113383'

In [11]:
params={"num_chars":len(chars), "lstm":True, "hidden":256}
mod = "/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/"
model = tf.estimator.Estimator(model_fn, mod, 
                               params=params)

model.train(lambda: train_inpt_fn(), steps=25000)
model.export_savedmodel("/media/joe/HD/text_pred_models/alice_lstm_256_adam_export/", serving_input_fn())

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4f828def0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints

INFO:tensorflow:global_step/sec: 5.18506
INFO:tensorflow:loss = 0.824451, step = 7100 (19.285 sec)
INFO:tensorflow:global_step/sec: 5.4589
INFO:tensorflow:loss = 0.840783, step = 7200 (18.318 sec)
INFO:tensorflow:global_step/sec: 5.54296
INFO:tensorflow:loss = 0.825914, step = 7300 (18.042 sec)
INFO:tensorflow:global_step/sec: 5.3646
INFO:tensorflow:loss = 0.774246, step = 7400 (18.642 sec)
INFO:tensorflow:global_step/sec: 5.28235
INFO:tensorflow:loss = 0.761384, step = 7500 (18.929 sec)
INFO:tensorflow:global_step/sec: 5.2092
INFO:tensorflow:loss = 0.714292, step = 7600 (19.198 sec)
INFO:tensorflow:global_step/sec: 5.34642
INFO:tensorflow:loss = 0.791309, step = 7700 (18.704 sec)
INFO:tensorflow:global_step/sec: 5.44468
INFO:tensorflow:loss = 0.765913, step = 7800 (18.372 sec)
INFO:tensorflow:global_step/sec: 5.31488
INFO:tensorflow:loss = 0.662816, step = 7900 (18.812 sec)
INFO:tensorflow:global_step/sec: 5.4367
INFO:tensorflow:loss = 0.7649, step = 8000 (18.394 sec)
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 5.20734
INFO:tensorflow:loss = 0.407981, step = 15100 (19.204 sec)
INFO:tensorflow:global_step/sec: 4.97977
INFO:tensorflow:loss = 0.417983, step = 15200 (20.081 sec)
INFO:tensorflow:global_step/sec: 4.86606
INFO:tensorflow:loss = 0.405588, step = 15300 (20.551 sec)
INFO:tensorflow:Saving checkpoints for 15319 into /media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/model.ckpt.


KeyboardInterrupt: 

In [12]:
model.export_savedmodel("/media/joe/HD/text_pred_models/alice_lstm_256_adam_export/", serving_input_fn())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['probs', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/model.ckpt-15319
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /media/joe/HD/text_pred_models/alice_lstm_256_adam_export/temp-b'1536116411'/saved_model.pb


b'/media/joe/HD/text_pred_models/alice_lstm_256_adam_export/1536116411'